In [1]:
import pyodbc
import pandas as pd
import numpy as np
from pprint import pprint
import datetime
from configparser import ConfigParser


In [2]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [3]:
# Code for Audit Integration
cursor.execute("""insert into FINCORE_DB.Fincore_Analytics.AUDIT_INTGN (JOB_RUN_ID,JOB_NAME,JOB_START_DATE,JOB_STATUS,TARGET_TBL,SOURCE_TBL,SOURCE_MAX_DATE)values
(COALESCE((select (max(JOB_RUN_ID)+1) from FINCORE_DB.Fincore_Analytics.AUDIT_INTGN),1), 'REPORT_RISK',GETDATE(),'Error','RISK_REPORT','RISK_TRANSACTION_SUMMARY_STG',(SELECT MAX(T.MAX_DATE) FROM (select MAX(CREATE_DATE) AS MAX_DATE from FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG UNION ALL select MAX(UPDATE_DATE) AS MAX_DATE from FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG )T))""")
conn.commit()
job_id = pd.read_sql_query("select max(JOB_RUN_ID) as job_run_id from FINCORE_DB.Fincore_Analytics.AUDIT_INTGN with(nolock) where job_name = 'REPORT_RISK' ",conn)
job_id = job_id.iloc[0,0]
rules_df = pd.read_sql_query("SELECT * FROM [FINCORE_DB].[Fincore_Analytics].[RISK_LKP_RULES] with(nolock)", conn)


In [4]:
store job_id

Stored 'job_id' (int64)


In [5]:
job_id

384

In [7]:
cursor.execute('''delete from [FINCORE_DB].[Fincore_Analytics].[RISK_REPORT_INCREMENTAL] ''')
conn.commit()

In [8]:
# POPULATING RISK_REPORT_INCREMENTAL
cursor.execute('''INSERT INTO [FINCORE_DB].[Fincore_Analytics].[RISK_REPORT_INCREMENTAL] 
(
 REPORT_ID
 )
SELECT  DISTINCT REPORT_ID FROM [FINCORE_DB].[Fincore_Analytics].RISK_TRANSACTION_SUMMARY_STG with (nolock) WHERE  
COALESCE (Update_Date,create_date) > COALESCE (( select MAX( SOURCE_MAX_DATE ) from FINCORE_DB.Fincore_Analytics.AUDIT_INTGN
where JOB_STATUS='Success' and JOB_NAME='REPORT_RISK' )
,cast('1900-01-01 00:00:00.0000000' as datetime2)) ''')
conn.commit()

In [9]:
count = pd.read_sql_query('''Select count(*) from [FINCORE_DB].[Fincore_Analytics].[RISK_REPORT_INCREMENTAL] with (nolock)''',conn)
count = count.iloc[0,0]

In [10]:
# EXITING THE CODE BY UPDATING AUDIT_INTGN IF THERE IS NO INCREMENTAL DATA.
if count == 0:
    cursor.execute(f'''Update FINCORE_DB.Fincore_Analytics.AUDIT_INTGN set JOB_END_DATE = Getdate(),JOB_STATUS = 'Success', NO_OF_RECORD = 0 ,SOURCE_MAX_DATE =  (SELECT MAX(T.MAX_DATE) FROM (SELECT MAX(CREATE_DATE) AS MAX_DATE FROM FINCORE_DB.FINCORE.PERSON_MASTER UNION SELECT MAX(Update_Date) AS MAX_DATE FROM FINCORE_DB.FINCORE.PERSON_MASTER) T) where  JOB_RUN_ID ={job_id}''')
    conn.commit()
    conn.close()
    raise SystemExit

In [11]:
# cursor.execute('''
# insert into FINCORE_DB.Fincore_Analytics.RISK_REPORT_SUMMARY_STG 
# select DISTINCT BATCH_ID,REPORT_ID,YEAR_REPORT,MONTH_REPORT,REPORT_TYPE,GETDATE() AS CREATE_DATE,NULL AS UPDATE_DATE,NULL AS TRAN_AMNT_CREDIT,NULL AS TRAN_AMNT_DEBIT from FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG_TRAN
# ''')
# conn.commit()

In [12]:
job_id

384

In [13]:
# cursor.execute('''INSERT INTO [FINCORE_DB].[Fincore_Analytics].[RISK_REPORT_SUMMARY_STG] 
# (
#  BATCH_ID,REPORT_ID,YEAR_REPORT,MONTH_REPORT,REPORT_TYPE
# )
# SELECT  DISTINCT BATCH_ID,REPORT_ID,YEAR_REPORT,MONTH_REPORT,REPORT_TYPE FROM [FINCORE_DB].[Fincore_Analytics].RISK_TRANSACTION_SUMMARY_STG with (nolock) WHERE  
# COALESCE (Update_Date,create_date) > COALESCE (( select MAX( SOURCE_MAX_DATE ) from FINCORE_DB.Fincore_Analytics.AUDIT_INTGN
# where JOB_STATUS='Success' and JOB_NAME='RISK_REPORT' )
# ,cast('1900-01-01 00:00:00.0000000' as datetime2)) ''')

In [14]:
cursor.execute('''INSERT INTO [FINCORE_DB].[Fincore_Analytics].[RISK_REPORT_SUMMARY_STG] 
(
 BATCH_ID,REPORT_ID,YEAR_REPORT,MONTH_REPORT,REPORT_TYPE
)
SELECT  DISTINCT BATCH_ID,A.REPORT_ID,YEAR_REPORT,MONTH_REPORT,REPORT_TYPE FROM
[FINCORE_DB].[Fincore_Analytics].RISK_TRANSACTION_SUMMARY_STG A with (nolock) 
inner join FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT_INCREMENTAL B WITH (NOLOCK)
ON A.REPORT_ID = B.REPORT_ID
''')
conn.commit()

In [15]:
tran_amnt_debit = '''UPDATE FINCORE_DB.Fincore_Analytics.RISK_REPORT_SUMMARY_STG  SET TRAN_AMNT_DEBIT  = A.TRAN_AMNT_DEBIT
FROM (
SELECT  A.REPORT_ID,SUM(B.TRAN_AMNT) AS TRAN_AMNT_DEBIT
FROM FINCORE_DB.Fincore_Analytics.RISK_REPORT_SUMMARY_STG A
INNER JOIN FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG_TRAN B
ON A.REPORT_ID = B.REPORT_ID
INNER JOIN [FINCORE_DB].[Fincore_Analytics].[RISK_REPORT_INCREMENTAL]  C
ON A.REPORT_ID = C.REPORT_ID
WHERE B.CR_DR_DEP_WITH IN ('DEBIT','Withdrawal','Buy','ATM Cash Withdrawal')
GROUP BY A.REPORT_ID
)a
WHERE a.REPORT_ID = FINCORE_DB.Fincore_Analytics.RISK_REPORT_SUMMARY_STG.REPORT_ID'''

In [16]:
tran_amnt_credit = '''
UPDATE FINCORE_DB.Fincore_Analytics.RISK_REPORT_SUMMARY_STG  SET TRAN_AMNT_CREDIT  = A.TRAN_AMNT_CREDIT
FROM (
SELECT  A.REPORT_ID,SUM(B.TRAN_AMNT) AS TRAN_AMNT_CREDIT
FROM FINCORE_DB.Fincore_Analytics.RISK_REPORT_SUMMARY_STG A
INNER JOIN FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG_TRAN B
ON A.REPORT_ID = B.REPORT_ID
INNER JOIN [FINCORE_DB].[Fincore_Analytics].[RISK_REPORT_INCREMENTAL]  C
ON A.REPORT_ID = C.REPORT_ID
WHERE (B.CR_DR_DEP_WITH IN ('Credit','Deposit','Sell') or CR_DR_DEP_WITH  IS NULL)
GROUP BY A.REPORT_ID
)a
WHERE a.REPORT_ID = FINCORE_DB.Fincore_Analytics.RISK_REPORT_SUMMARY_STG.REPORT_ID 
'''

In [17]:
cursor.execute(tran_amnt_credit)
cursor.execute(tran_amnt_debit)
conn.commit()

In [18]:
rules_df = pd.read_sql_query('''SELECT * FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_LKP_RULES WITH (NOLOCK)''',conn)

In [19]:
rules_df = rules_df[rules_df['PERSON_TYPE'] == 'Report']

In [20]:
rules_df.head()

RULE_ID     RULE_NO                                          RULE_DESC  \
52    5.25  R005025000  Donation from a single entity is greater than ...   
54     5.1  R005001000  Any of reported parties already have trigerred...   
56     5.2  R005002000  Untraceable flags identified (E.g. true accoun...   
57     5.3  R005003000  GoS tags reveals  high risk tags on any of ent...   
58     5.4  R005004000  High Risk Purpose code reported in CBWTR\r\n\r...   

   PERSON_TYPE         RISK_TYPE  IS_ACTIVE_FLAG CREATE_DATE UPDATE_DATE  \
52      Report       Report Risk               0  2022-12-07  1900-01-01   
54      Report  Auto High Rules                1  2022-12-07  1900-01-01   
56      Report   Auto High Rules               0  2022-12-07  1900-01-01   
57      Report       Report Risk               1  2022-12-07  1900-01-01   
58      Report       Report Risk               1  2022-12-07  1900-01-01   

   SOURCE_DATA_EXISTING_UNDRSTD     DEVELOPMENT_STATUS CODE_REVIEW_STATUS  \
52                         NULL            Not Started               NULL   
54                         NULL  Development Completed               NULL   
56                         NULL            Not Started               NULL   
57                         NULL            Not Started               NULL   
58                         NULL            Not Started               NULL   

   DEPENDENCY  WEIGHTAGE  PRIORITY_NO  \
52       NULL         10          4.0   
54       NULL          9          4.0   
56       NULL          8          4.0   
57       NULL          8          4.0   
58       NULL         10          4.0   

                                                Query QUERY_TEMP  \
52                                               NULL       None   
54  Merge into fincore_db.Fincore_Analytics.risk_r...       None   
56                                               NULL       None   
57  \r\n   Merge into fincore_db.Fincore_Analytics...       None   
58  \r\n   Merge into fincore_db.Fincore_Analytics...       None   

    RISK_SCORE_STATIC_FLAG  
52                     0.0  
54                     0.0  
56                     0.0  
57                     1.0  
58                     1.0

In [21]:
for query in rules_df[(rules_df['IS_ACTIVE_FLAG'] == 1) ]['Query'].to_list():
    pprint(query)
    cursor.execute(query)
    print("Done")
#     print(namestr(query,globals())[0]," DONE!")
    conn.commit()
    


('Merge into fincore_db.Fincore_Analytics.risk_report_rules with (holdlock) as '
 'target using      \r\n'
 '\t  (\r\n'
 '\t  \r\n'
 '\t  \r\n'
 '\t  SELECT          distinct rtsse.BATCH_ID,           '
 'rtsse.REPORT_ID,       \r\n'
 "\t  'R005001000' as RULE_NO,        rpss.organization_MASTER_ID as ENTITY_ID "
 ',  \r\n'
 '\t  10 AS RISK_SCORE_STATIC,  \r\n'
 '\t  GETDATE() AS CREATE_DATE   \r\n'
 '\t  FROM fincore_db.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG_ENTITY '
 'rtsse with (nolock)      \r\n'
 '\t  inner join '
 'FINCORE_DB.Fincore_Analytics.RISK_organization_SUMMARY_STG_RULES rpss  with '
 '(nolock)       \r\n'
 '\t  on rtsse.ENTITY_master_ID=rpss.ORGANIZATION_MASTER_ID  where   '
 "rtsse.Entity_type = 'O'\r\n"
 '\t  and rpss.rule_no in (select rule_no from '
 'fincore_db.fincore_analytics.risk_lkp_rules where \r\n'
 "\t  risk_type='Auto High Rules ')\r\n"
 '\t  \r\n'
 '\t  \r\n'
 '\t  union              \r\n'
 '\t  \r\n'
 '\t  SELECT    distinct            rtsse.BATCH

Done
('\r\n'
 ' Merge into fincore_db.Fincore_Analytics.risk_report_rules with '
 '(holdlock) \r\n'
 '  as target using \r\n'
 '\r\n'
 '(SELECT \r\n'
 'DISTINCT rtss.BATCH_ID,\r\n'
 'rtss.REPORT_ID, \r\n'
 "'R005014000' as RULE_NO,\r\n"
 "'Y' AS RULE_VALUE, \r\n"
 '8 AS RISK_SCORE_STATIC,       \r\n'
 'GETDATE() AS CREATE_DATE        \r\n'
 'FROM fincore_db.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG '
 'rtss          \r\n'
 'inner join fincore_db.fincore.Cps_Report_Details crd  on '
 'crd.Fiu_Batch_Id=rtss.BATCH_ID and crd.Report_Id=rtss.REPORT_ID         \r\n'
 'INNER join fincore_bridge_db.Fincore_bridge.fingate_gos fggos on '
 'crd.Batch_Id=fggos.batchid and crd.Report_Id=fggos.reportid        \r\n'
 "WHERE fggos.kycFundSource='Unknown' ) as source\r\n"
 ' on target.BATCH_ID =source.BATCH_ID and target.REPORT_ID=source.REPORT_ID '
 'and \r\n'
 "  target.RULE_NO = 'R005014000'     \r\n"
 '  and target.rule_value=source.RULE_VALUE when matched then update set   \r\n'
 '  '
 'targe

In [22]:

risk_lkp = pd.read_sql_query("SELECT * FROM [FINCORE_DB].[Fincore_Analytics].[RISK_LKP_RULES] ", conn)
static_rules = risk_lkp[(risk_lkp['RISK_SCORE_STATIC_FLAG'] == 1) & (risk_lkp['IS_ACTIVE_FLAG'] == 1) & (risk_lkp['PERSON_TYPE'] == 'Report') & (risk_lkp['RISK_TYPE'] != 'Auto High Rules')]['RULE_NO'].to_list()
auto_high_rules =  risk_lkp[(risk_lkp['IS_ACTIVE_FLAG'] == 1) & (risk_lkp['RISK_TYPE'] == 'Auto High Rules ')]['RULE_NO'].to_list()
non_stat_rules = risk_lkp[(risk_lkp['RISK_SCORE_STATIC_FLAG'] == 0) & (risk_lkp['IS_ACTIVE_FLAG'] == 1) & (risk_lkp['PERSON_TYPE'] == 'Report')& (risk_lkp['RISK_TYPE'] != 'Auto High Rules')]['RULE_NO'].to_list()



In [23]:
auto_high_rules

['R005001000', 'P001005000', 'O001006000']

In [24]:
# Read risk_person_summary_stg_rules for the static rules
if len(auto_high_rules) == 1:
    report_rules = pd.read_sql_query("SELECT A.* FROM [Fincore_db].[Fincore_Analytics].[RISK_REPORT_RULES] A INNER JOIN [FINCORE_DB].[Fincore_Analytics].[RISK_REPORT_INCREMENTAL] B ON A.REPORT_ID = B.REPORT_ID where RULE_NO in ('{}')".format(auto_high_rules[0]),conn)
    
elif(len(auto_high_rules)== 0):
    report_rules = pd.read_sql_query("SELECT A.* FROM [Fincore_db].[Fincore_Analytics].[RISK_REPORT_RULES]  A INNER JOIN [FINCORE_DB].[Fincore_Analytics].[RISK_REPORT_INCREMENTAL] B ON A.REPORT_ID = B.REPORT_ID  where RULE_NO in ('')",conn)
    
else:
    auto_high_rules = tuple(auto_high_rules)
    report_rules = pd.read_sql_query("SELECT A.* FROM [Fincore_db].[Fincore_Analytics].[RISK_REPORT_RULES]  A INNER JOIN [FINCORE_DB].[Fincore_Analytics].[RISK_REPORT_INCREMENTAL] B ON A.REPORT_ID = B.REPORT_ID  where RULE_NO in" + str (auto_high_rules) , conn)

# stat_rule_df = pd.read_sql_query("SELECT * FROM [FINCORE_DB].[Fincore_Analytics].[RISK_PERSON_SUMMARY_STG_RULES] where RULE_NO in "+str(tuple(static_rules)) , conn)


In [25]:
report_rules

Empty DataFrame
Columns: [BATCH_ID, REPORT_ID, RULE_NO, RULE_VALUE, RISK_SCORE_STATIC, CREATE_DATE]
Index: []

In [26]:
report_rules_auto_high = report_rules['REPORT_ID'].values.tolist()

In [27]:
report_rules_auto_high

[]

In [28]:
# for report in report_rules_auto_high:
#     print('''INSERT INTO FINCORE_DB.Fincore_Analytics.RISK_REPORT
#   select distinct  A.BATCH_ID,A.REPORT_ID,C.REPORT_TYPE,10 as RISK_SCORE,
#   GETDATE () AS CREATE_DATE, NULL AS RISK_SCORE_QUERY,NULL AS RISK_SCORE_WEIGHTED_AVG
# from FINCORE_DB.Fincore_Analytics.RISK_REPORT_RULES A
# inner join FINCORE_DB.Fincore_Analytics.RISK_REPORT_SUMMARY_STG C
# on C.REPORT_ID = A.REPORT_ID where C.REPORT_ID = {}  '''.format(report_rules_auto_high) )
#     conn.commit()
# #     conn.close()

In [29]:
if len(report_rules_auto_high) == 1:
    
    cursor.execute('''INSERT INTO FINCORE_DB.Fincore_Analytics.RISK_REPORT
select A.BATCH_ID,A.REPORT_ID,REPORT_TYPE,10 as RISK_SCORE,
GETDATE () AS CREATE_DATE,
NULL as RISK_SCORE_QUERY,NULL AS RISK_SCORE_WEIGHTED_AVG
from FINCORE_DB.Fincore_Analytics.RISK_REPORT_RULES A
inner join FINCORE_DB.Fincore_Analytics.RISK_REPORT_SUMMARY_STG C
on C.REPORT_ID = A.REPORT_ID 
inner join FINCORE_DB.Fincore_Analytics.RISK_LKP_RULES B
on A.RULE_NO = B.RULE_NO
where A.REPORT_ID = {}
GROUP BY A.REPORT_ID,A.BATCH_ID,REPORT_TYPE '''.format(report_rules_auto_high[0]))
    conn.commit()
    
    
    
elif len(report_rules_auto_high) > 1 :
    report_rules_auto_high = tuple(report_rules_auto_high)
    cursor.execute('''INSERT INTO FINCORE_DB.Fincore_Analytics.RISK_REPORT
select A.BATCH_ID,A.REPORT_ID,REPORT_TYPE,10 AS RISK_SCORE,
GETDATE() AS CREATE_DATE,
NULL as RISK_SCORE_QUERY,NULL AS RISK_SCORE_WEIGHTED_AVG
from FINCORE_DB.Fincore_Analytics.RISK_REPORT_RULES A
inner join FINCORE_DB.Fincore_Analytics.RISK_REPORT_SUMMARY_STG C
on C.REPORT_ID = A.REPORT_ID 
inner join FINCORE_DB.Fincore_Analytics.RISK_LKP_RULES B
on A.RULE_NO = B.RULE_NO
where A.REPORT_ID  IN {} 
GROUP BY A.REPORT_ID,A.BATCH_ID,REPORT_TYPE'''.format(report_rules_auto_high))
    conn.commit()
    

In [30]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [31]:
# non_stat_rules = tuple(non_stat_rules)
stat_rules = tuple(static_rules)

In [32]:
# Read risk_person_summary_stg_rules for the static rules
if len(non_stat_rules) == 1:
    report_rules = pd.read_sql_query("SELECT A.* FROM [Fincore_db].[Fincore_Analytics].[RISK_REPORT_RULES] A INNER JOIN [FINCORE_DB].[Fincore_Analytics].[RISK_REPORT_INCREMENTAL] B ON A.REPORT_ID = B.REPORT_ID where RULE_NO in ('{}')".format(non_stat_rules[0]),conn)
    
elif(len(non_stat_rules)== 0):
    report_rules = pd.read_sql_query("SELECT A.* FROM [Fincore_db].[Fincore_Analytics].[RISK_REPORT_RULES] A INNER JOIN [FINCORE_DB].[Fincore_Analytics].[RISK_REPORT_INCREMENTAL] B ON A.REPORT_ID = B.REPORT_ID where RULE_NO in ('')",conn)
    
else:
    non_stat_rules = tuple(non_stat_rules)
    report_rules = pd.read_sql_query("SELECT A.* FROM [Fincore_db].[Fincore_Analytics].[RISK_REPORT_RULES] A INNER JOIN [FINCORE_DB].[Fincore_Analytics].[RISK_REPORT_INCREMENTAL] B ON A.REPORT_ID = B.REPORT_ID where RULE_NO in" + str (non_stat_rules) , conn)

# stat_rule_df = pd.read_sql_query("SELECT * FROM [FINCORE_DB].[Fincore_Analytics].[RISK_PERSON_SUMMARY_STG_RULES] where RULE_NO in "+str(tuple(static_rules)) , conn)


In [33]:
report_rules

BATCH_ID  REPORT_ID     RULE_NO  RULE_VALUE RISK_SCORE_STATIC  \
0  2022122710111177     113121  R005005000  4450000.00              None   
1  2022122710111177     113121  R005006000           1              None   
2  2022122710111177     113121  R005007000           2              None   

  CREATE_DATE  
0  2023-03-03  
1  2023-03-03  
2  2023-03-03

In [34]:
report_rules = report_rules[~report_rules['REPORT_ID'].isin([1,2])]

In [35]:
report_rules = report_rules[~report_rules['REPORT_ID'].isin(report_rules_auto_high)]

In [36]:
summary_stg = pd.read_sql_query('''select A.* from fincore_db.fincore_analytics.risk_report_summary_stg A INNER JOIN [FINCORE_DB].[Fincore_Analytics].[RISK_REPORT_INCREMENTAL] B ON A.REPORT_ID = B.REPORT_ID ''',conn)

In [37]:
summary_stg

BATCH_ID  REPORT_ID  YEAR_REPORT  MONTH_REPORT REPORT_TYPE  \
0  2022122710111177     113121         2022            12         STR   

  CREATE_DATE UPDATE_DATE  TRAN_AMNT_CREDIT TRAN_AMNT_DEBIT  
0        None        None         4450000.0            None

In [38]:
summary_stg = summary_stg[['REPORT_ID','REPORT_TYPE']]

In [52]:
cursor.execute('''WITH CTE AS (
select
A.REPORT_ID,
A.RULE_NO,
coalesce((case 
	when c.THRESHOLD1_1 IS null then 0
	when A.RULE_VALUE < c.threshold1_1 then 0 
	when A.RULE_VALUE = c.threshold1_1 then 1
	when A.RULE_VALUE <= c.THRESHOLD1_2 then 2
	when A.RULE_VALUE <= c.THRESHOLD1_3 then 3
	when A.RULE_VALUE <= c.THRESHOLD1_4 then 4
	when A.RULE_VALUE <= c.THRESHOLD1_5 then 5
	when A.RULE_VALUE <= c.THRESHOLD1_6 then 6
	when A.RULE_VALUE <= c.THRESHOLD1_7 then 7
	when A.RULE_VALUE <= c.THRESHOLD1_8 then 8
	when A.RULE_VALUE <= c.THRESHOLD1_9 then 9
	when A.RULE_VALUE <= c.THRESHOLD1_10 then 10
	else 10
	end),1) RISK_SCORE,
	GETDATE() AS CREATE_DATE
from FINCORE_DB.Fincore_Analytics.RISK_REPORT_RULES a 
inner join FINCORE_DB.Fincore_Analytics.RISK_REPORT_SUMMARY_STG b 
on a.REPORT_ID = b.REPORT_ID
left join FINCORE_DB.Fincore_Analytics.RISK_lkp_entity_threshold c on 
c.PEER_GROUP_ID = b.REPORT_TYPE and a.RULE_NO = c.RULE_NO
inner join FINCORE_DB.Fincore_Analytics.RISK_REPORT_incremental d
on a.report_id = d.report_id
inner join FINCORE_DB.Fincore_Analytics.risk_lkp_rules rlr with (nolock)
on rlr.rule_no = a.rule_no
where rlr.RISK_SCORE_STATIC_flag = 0)

UPDATE  FINCORE_DB.Fincore_Analytics.RISK_REPORT_RULES SET RISK_SCORE_STATIC = CTE.RISK_SCORE
FROM FINCORE_DB.Fincore_Analytics.RISK_REPORT_RULES A
INNER JOIN CTE
ON CTE.REPORT_ID = A.REPORT_ID AND CTE.RULE_NO = A.RULE_NO''')
conn.commit()

In [53]:
if len(report_rules_auto_high) == 0:
    cursor.execute('''INSERT INTO FINCORE_DB.Fincore_Analytics.RISK_REPORT
select A.BATCH_ID,A.REPORT_ID,REPORT_TYPE,NULL,
GETDATE () AS CREATE_DATE,round(CAST(SUM(B.WEIGHTAGE*A.RISK_SCORE_STATIC) AS float)/CAST(SUM(B.WEIGHTAGE) AS float),0) AS RISK_SCORE_QUERY,NULL
from FINCORE_DB.Fincore_Analytics.RISK_REPORT_RULES A
inner join FINCORE_DB.Fincore_Analytics.RISK_REPORT_SUMMARY_STG C
on C.REPORT_ID = A.REPORT_ID 
INNER JOIN FINCORE_DB.Fincore_Analytics.RISK_REPORT_INCREMENTAL D
ON A.REPORT_ID = D.REPORT_ID
inner join FINCORE_DB.Fincore_Analytics.RISK_LKP_RULES B
on A.RULE_NO = B.RULE_NO
GROUP BY A.REPORT_ID,A.BATCH_ID,REPORT_TYPE''')
    conn.commit()
    
elif len(report_rules_auto_high) == 1:
    
    cursor.execute('''INSERT INTO FINCORE_DB.Fincore_Analytics.RISK_REPORT
select A.BATCH_ID,A.REPORT_ID,REPORT_TYPE,NULL,
GETDATE () AS CREATE_DATE,
round(CAST(SUM(B.WEIGHTAGE*A.RISK_SCORE_STATIC) AS float)/CAST(SUM(B.WEIGHTAGE) AS float),0) AS RISK_SCORE_QUERY,NULL
from FINCORE_DB.Fincore_Analytics.RISK_REPORT_RULES A
inner join FINCORE_DB.Fincore_Analytics.RISK_REPORT_SUMMARY_STG C
on C.REPORT_ID = A.REPORT_ID 
INNER JOIN FINCORE_DB.Fincore_Analytics.RISK_REPORT_INCREMENTAL D
ON A.REPORT_ID = D.REPORT_ID
inner join FINCORE_DB.Fincore_Analytics.RISK_LKP_RULES B
on A.RULE_NO = B.RULE_NO
where A.REPORT_ID <> {}
GROUP BY A.REPORT_ID,A.BATCH_ID,REPORT_TYPE '''.format(report_rules_auto_high[0]))
    conn.commit()
    
    
else:
    report_rules_auto_high = tuple(report_rules_auto_high)
    cursor.execute('''INSERT INTO FINCORE_DB.Fincore_Analytics.RISK_REPORT
select A.BATCH_ID,A.REPORT_ID,REPORT_TYPE,NULL,
GETDATE () AS CREATE_DATE,round(CAST(SUM(B.WEIGHTAGE*A.RISK_SCORE_STATIC) AS float)/CAST(SUM(B.WEIGHTAGE) AS float),0) AS RISK_SCORE_QUERY,NULL
from FINCORE_DB.Fincore_Analytics.RISK_REPORT_RULES A
inner join FINCORE_DB.Fincore_Analytics.RISK_REPORT_SUMMARY_STG C
on C.REPORT_ID = A.REPORT_ID 
INNER JOIN FINCORE_DB.Fincore_Analytics.RISK_REPORT_INCREMENTAL D
ON A.REPORT_ID = D.REPORT_ID
inner join FINCORE_DB.Fincore_Analytics.RISK_LKP_RULES B
on A.RULE_NO = B.RULE_NO
where A.REPORT_ID NOT IN {} 
GROUP BY A.REPORT_ID,A.BATCH_ID,REPORT_TYPE '''.format(report_rules_auto_high))
    conn.commit()
    

In [ ]:
conn.close()